In [71]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [72]:
pointsrdd = sc.textFile("./KMeans/points.txt")

In [73]:
pointsrdd_split = pointsrdd.map(lambda line: line.split('\t'))

In [74]:
#pointsrdd_split.collect()
#pointsrdd_split.collect()[1][1]
#pointsrdd_split.collect()

In [92]:
centroids1 = [[2,5],[6,2],[1,1]]
#print(centroids1[2])

In [93]:
# function to calculate distance between 2 co-ordinates
def dist(x1,y1,x2,y2):
    return ((((x2-x1)**2) + ((y2-y1)**2))**0.5)

In [94]:
# function to assign centroids based on distance
def mymap1(x1,y1,c1):
    centroidx1 = c1[0][0]
    centroidy1 = c1[0][1]
    centroidx2 = c1[1][0]
    centroidy2 = c1[1][1]
    centroidx3 = c1[2][0]
    centroidy3 = c1[2][1]
    minDist = dist(x1,y1,centroidx1,centroidy1)
    closestCentroid = (centroidx1,centroidy1)
    #print(minDist)
    minDist1 = dist(x1,y1,centroidx2,centroidy2)
    if(minDist1 < minDist):
        minDist = minDist1
        closestCentroid = (centroidx2,centroidy2)
    minDist2 = dist(x1,y1,centroidx3,centroidy3)
    if(minDist2 < minDist):
        minDist = minDist2
        closestCentroid = (centroidx3,centroidy3)
    return(closestCentroid)

In [95]:
# simple test
mymap1(2,4,centroids1)

(2, 5)

In [78]:
#centroids.collect()
pointsrdd_split1 = pointsrdd_split.map(lambda x: (x[0],x[1]))
#pointsrdd_split1.collect()
pointsrdd_split1.take(3)
#pointsrdd_split1.map(lambda x: (x[0])).collect()

[('3.023', '5.138'), ('3.075', '4.989'), ('2.321', '5.350')]

In [79]:
pointsrdd_split_centroidmap2 = pointsrdd_split1.map(lambda x: (mymap1(float(x[0]),float(x[1]),centroids1),(x[0],x[1])))
                                               #.sortByKey()                                            

In [80]:
# original centroid assignment
pointsrdd_split_centroidmap2.take(3)

[((2, 5), ('3.023', '5.138')),
 ((2, 5), ('3.075', '4.989')),
 ((2, 5), ('2.321', '5.350'))]

In [81]:
# new centroid calculation
#centroid_new = pointsrdd_split_centroidmap2.reduceByKey(lambda x,y: ((float(x[0])+float(y[0]))/2, (float(x[1])+float(y[1]))/2)) \
#                                           .sortByKey()
centroid_new = pointsrdd_split_centroidmap2.reduceByKey(lambda x,y: (round((float(x[0])+float(y[0])),3)/2, round((float(x[1])+float(y[1])),3)/2))


# new centroids after 1st iteration 
centroid_new.collect()

[((1, 1), (2.153, 1.1925)), ((6, 2), (4.788, 1.936)), ((2, 5), (3.004, 4.908))]

In [82]:
print("New centroids after 1st iteration")
centroids1[0] = centroid_new.collect()[0][1]
centroids1[1] = centroid_new.collect()[1][1]
centroids1[2] = centroid_new.collect()[2][1]
print(centroids1)

New centroids after 1st iteration
[(2.153, 1.1925), (4.788, 1.936), (3.004, 4.908)]


In [88]:
for i in range(9):
    myMapCalc = pointsrdd_split1.map(lambda x: (mymap1(float(x[0]),float(x[1]),centroids1),(x[0],x[1])))
    myReduceCalc = myMapCalc.reduceByKey(lambda x,y: (round((float(x[0])+float(y[0])),3)/2, round((float(x[1])+float(y[1])),3)/2))
                                            

    centroids1[0] = myReduceCalc.collect()[0][1]
    centroids1[1] = myReduceCalc.collect()[1][1]
    centroids1[2] = myReduceCalc.collect()[2][1]
    
#myMapCalc_set1 = myMapCalc.filter(lambda x: x[0] = (2,1))
print("New centroids after 10 iterations are: ")
print(centroids1)
#print(myReduceCalc.collect())
    

New corresponding centroids after 20 iterations are: 
[(3.004, 4.908), (2.153, 1.1925), (4.788, 1.936)]


In [89]:
final_dataset = myReduceCalc.join(pointsrdd_split_centroidmap2)
#final_dataset.take(5)

In [90]:
final_dataset2 = final_dataset.map(lambda x: (x[1][0],x[1][1]))

In [91]:
print("Final output with format--> centroid::PointCoordinate1#PointCoordinate2")
final_dataset2.map(lambda x: (x[0],"::",(x[1][0] + "#" + x[1][1]))).collect()

Final output with format --> centroid::PointCoordinate1#PointCoordinate2


[((3.004, 4.908), '::', '3.023#5.138'),
 ((3.004, 4.908), '::', '3.075#4.989'),
 ((3.004, 4.908), '::', '2.321#5.350'),
 ((3.004, 4.908), '::', '3.328#4.944'),
 ((3.004, 4.908), '::', '3.195#5.186'),
 ((3.004, 4.908), '::', '3.034#4.656'),
 ((3.004, 4.908), '::', '3.484#4.877'),
 ((3.004, 4.908), '::', '3.023#4.951'),
 ((3.004, 4.908), '::', '2.707#4.871'),
 ((3.004, 4.908), '::', '2.894#5.031'),
 ((3.004, 4.908), '::', '2.880#5.207'),
 ((3.004, 4.908), '::', '2.780#5.196'),
 ((3.004, 4.908), '::', '2.941#4.951'),
 ((3.004, 4.908), '::', '3.213#5.123'),
 ((3.004, 4.908), '::', '2.972#4.941'),
 ((3.004, 4.908), '::', '3.138#5.460'),
 ((3.004, 4.908), '::', '3.247#5.185'),
 ((3.004, 4.908), '::', '3.602#5.405'),
 ((3.004, 4.908), '::', '2.857#4.686'),
 ((3.004, 4.908), '::', '3.390#4.801'),
 ((3.004, 4.908), '::', '3.125#4.809'),
 ((3.004, 4.908), '::', '3.363#5.073'),
 ((3.004, 4.908), '::', '2.762#4.868'),
 ((3.004, 4.908), '::', '2.798#5.120'),
 ((3.004, 4.908), '::', '3.320#4.969'),
